**Проект выполнен полностью с использованием языка SQL**

1. Посчитайте, сколько компаний закрылось.

In [ ]:
SELECT COUNT(*)
FROM company
WHERE status = 'closed'

2. Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total .

In [ ]:
SELECT funding_total
FROM company
WHERE country_code = 'USA' AND category_code = 'news'
ORDER BY funding_total DESC;

3. Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash'
 AND acquired_at BETWEEN '2011-01-01' AND '2013-12-31';

4. Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.

In [ ]:
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%';

5. Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'.

In [ ]:
SELECT *
FROM people
WHERE twitter_username LIKE '%money%' 
  AND last_name LIKE 'K%';

6. Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
SELECT country_code AS country,
       SUM(funding_total) AS total_money
FROM company
GROUP BY country
ORDER BY total_money DESC;

7. Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
SELECT funded_at,
       MIN(raised_amount),
       MAX(raised_amount)
FROM funding_round
GROUP BY funded_at
HAVING NOT MIN(raised_amount) = '0' 
   AND NOT MIN(raised_amount) =  MAX(raised_amount);

8. Создайте поле с категориями:  
Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.  
Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.  
Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.  
Отобразите все поля таблицы fund и новое поле с категориями.

In [ ]:
SELECT *,
  CASE
      WHEN invested_companies >= 100 THEN 'high_activity'
      WHEN invested_companies >=20 AND invested_companies < 100 THEN 'middle_activity'
      WHEN invested_companies < 20 THEN 'low_activity'
  END
FROM fund;

9. Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие.   
Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
SELECT CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       ROUND(AVG(investment_rounds))
FROM fund
GROUP BY activity
ORDER BY ROUND(AVG(investment_rounds))

10. Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы.     
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. Выгрузите десять самых активных стран-инвесторов.  
Отсортируйте таблицу по среднему количеству компаний от большего к меньшему, а затем по коду страны в лексикографическом порядке.

In [ ]:
SELECT country_code,
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies)
FROM fund
WHERE founded_at BETWEEN '2010-01-01' AND '2012-12-31'
GROUP BY country_code
HAVING NOT MIN(invested_companies) = 0
ORDER BY AVG(invested_companies) DESC, country_code
LIMIT 10;

11. Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
SELECT p.first_name,
       p.last_name,
       e.instituition
FROM people AS p
LEFT JOIN education AS e ON p.id = e.person_id;

12. Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
SELECT c.name AS name_company,
       COUNT(DISTINCT e.instituition) AS amount
FROM company AS c
JOIN people AS p ON c.id = p.company_id
JOIN education AS e ON p.id = e.person_id
GROUP BY c.name
ORDER BY amount DESC
LIMIT 5;

13. Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
SELECT DISTINCT c.name
FROM company AS c
JOIN funding_round AS fr ON c.id = fr.company_id
WHERE c.status = 'closed' AND fr.id IN 
                                       (SELECT id
                                        FROM funding_round
                                        WHERE is_first_round = '1'
                                         AND is_last_round = '1');

14. Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
SELECT p.id
FROM people AS p
INNER JOIN company AS c ON p.company_id = c.id
WHERE c.name IN 
                (SELECT DISTINCT c.name
                 FROM company AS c
                 JOIN funding_round AS fr ON c.id = fr.company_id
                 WHERE c.status = 'closed' AND fr.id IN 
                                                       (SELECT id
                                                        FROM funding_round
                                                        WHERE is_first_round = '1'
                                                        AND is_last_round = '1'));

15. Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
WITH 

uniq_id_people AS (SELECT p.id AS id
                   FROM people AS p
                   INNER JOIN company AS c ON p.company_id = c.id
                   WHERE c.name IN (SELECT DISTINCT c.name
                                    FROM company AS c
                                    JOIN funding_round AS fr ON c.id = fr.company_id
                                    WHERE c.status = 'closed' AND fr.id IN (SELECT id
                                                                            FROM funding_round
                                                                            WHERE is_first_round = '1'
                                                                            AND is_last_round = '1')))
                   
SELECT DISTINCT uniq_id_people.id,
      e.instituition
FROM uniq_id_people INNER JOIN education AS e ON uniq_id_people.id = e.person_id;

16. Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
WITH 

uniq_id_people AS (SELECT p.id AS id
                   FROM people AS p
                   INNER JOIN company AS c ON p.company_id = c.id
                   WHERE c.name IN (SELECT DISTINCT c.name
                                    FROM company AS c
                                    JOIN funding_round AS fr ON c.id = fr.company_id
                                    WHERE c.status = 'closed' AND fr.id IN (SELECT id
                                                                            FROM funding_round
                                                                            WHERE is_first_round = '1'
                                                                            AND is_last_round = '1')))
                   
SELECT DISTINCT uniq_id_people.id,
      COUNT(e.instituition)
FROM uniq_id_people INNER JOIN education AS e ON uniq_id_people.id = e.person_id
GROUP BY uniq_id_people.id;

17. Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
SELECT COUNT(e.instituition) * 1.0 / COUNT(DISTINCT person_id)
FROM education AS e
WHERE person_id IN 
                   (SELECT p.id AS id
                   FROM people AS p
                   INNER JOIN company AS c ON p.company_id = c.id
                   WHERE c.name IN (SELECT DISTINCT c.name
                                    FROM company AS c
                                    JOIN funding_round AS fr ON c.id = fr.company_id
                                    WHERE c.status = 'closed' AND fr.id IN (SELECT id
                                                                            FROM funding_round
                                                                            WHERE is_first_round = '1'
                                                                            AND is_last_round = '1')));

18. Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.
*(сервис, запрещённый на территории РФ)

In [ ]:
WITH 

a AS (SELECT DISTINCT p.id
      FROM people AS p
      INNER JOIN company AS c ON c.id = p.company_id
      WHERE c.name = 'Facebook'),
b AS (SELECT COUNT(e.instituition) AS amount,
      p.id
      FROM people AS p 
      INNER JOIN education AS e ON p.id = e.person_id
      GROUP BY p.id)
      
SELECT AVG(amount)
FROM a
INNER JOIN b ON a.id = b.id;

19. Составьте таблицу из полей:  
name_of_fund — название фонда;  
name_of_company — название компании;  
amount — сумма инвестиций, которую привлекла компания в раунде.  
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
SELECT f.name AS name_of_fund,
       st.name AS name_of_company,
       fr.raised_amount AS amount
FROM investment AS i
LEFT JOIN fund AS f ON f.id = i.fund_id
LEFT JOIN company AS st ON st.id = i.company_id
INNER JOIN 
           (SELECT *
            FROM funding_round 
            WHERE funded_at BETWEEN '2012-01-01' AND '2013-12-31') AS fr
ON fr.id = i.funding_round_id 
WHERE st.milestones > 6;


20. Выгрузите таблицу, в которой будут такие поля:  
- название компании-покупателя;  
- сумма сделки;  
- название компании, которую купили;  
- сумма инвестиций, вложенных в купленную компанию;  
доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.  
Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке.  
Ограничьте таблицу первыми десятью записями.

In [ ]:
SELECT c.name AS ascuiring_company,
       ac.price_amount,
       c_1.name AS ascuired_company,
       c_1.funding_total,
       ROUND(ac.price_amount/c_1.funding_total) AS percent
FROM acquisition AS ac
LEFT JOIN company AS c ON ac.acquiring_company_id = c.id
LEFT JOIN company AS c_1 ON ac.acquired_company_id = c_1.id
WHERE ac.price_amount > 0 AND c_1.funding_total > 0
ORDER BY ac.price_amount DESC, ascuired_company
LIMIT 10;

21. Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
SELECT c.name, 
       EXTRACT(MONTH FROM CAST(f.funded_at AS date)) AS month
FROM company AS c
LEFT JOIN funding_round AS f ON c.id = f.company_id
WHERE f.funded_at BETWEEN '2010-01-01' AND '2013-12-31' 
      AND c.category_code = 'social'
      AND f.raised_amount > 0;

22. Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:  
- номер месяца, в котором проходили раунды;  
- количество уникальных названий фондов из США, которые инвестировали в этом месяце;  
- количество компаний, купленных за этот месяц;  
- общая сумма сделок по покупкам в этом месяце.

In [ ]:
WITH 
data_month AS (SELECT EXTRACT(MONTH FROM CAST(fr.funded_at AS date)) AS month_1,
               COUNT(DISTINCT f.name) AS amount_company_1
               FROM funding_round AS fr
               JOIN investment AS i ON fr.id = i.funding_round_id
               JOIN fund AS f ON i.fund_id = f.id
               WHERE EXTRACT(YEAR FROM CAST(fr.funded_at AS date)) BETWEEN '2010' AND '2013'
                  AND f.country_code = 'USA'
               GROUP BY month_1),
               
info_acquisition AS (SELECT EXTRACT(MONTH FROM CAST(a.acquired_at AS date)) AS month_1,
                     COUNT(a.acquired_company_id) AS amount_buy_company_1,
                     SUM(a.price_amount) AS sum_price_1
                     FROM acquisition AS a 
                     WHERE EXTRACT(YEAR FROM CAST(a.acquired_at AS date)) BETWEEN '2010' AND '2013'
                     GROUP BY month_1)

SELECT data_month.month_1 AS month,
       data_month.amount_company_1 AS amount_company,
       info_acquisition.amount_buy_company_1 AS amount_buy_company,
       info_acquisition.sum_price_1 AS sum_price
FROM data_month LEFT JOIN info_acquisition ON data_month.month_1 = info_acquisition.month_1;

23. Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH 
one AS (SELECT country_code AS country,
            AVG(funding_total) AS year_2011
        FROM company
        WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2011
        GROUP BY country),

two AS (SELECT country_code AS country,
            AVG(funding_total) AS year_2012
        FROM company
        WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2012
        GROUP BY country),
       
three AS (SELECT country_code AS country,
            AVG(funding_total) AS year_2013
        FROM company
        WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2013
        GROUP BY country)        

SELECT one.country,
       one.year_2011,
       two.year_2012,
       three.year_2013
FROM one INNER JOIN two ON one.country = two.country
INNER JOIN three ON two.country = three.country
ORDER BY year_2011 DESC;